## Exploring linear probing classification using the DINO class token as the feature space

In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import f1_score, classification_report, confusion_matrix

from model_lora_vit import get_vit, load_lora_vit_from_dino_ckpt
from data_transforms import get_random_transform, get_deterministic_transform
from dataloader_tmed import TMED2

torch.hub.set_dir("../pretrained_weights")

In [2]:
# configure the GPU
device = 3 if torch.cuda.is_available() else "cpu"

# configure batch size for training
batch_size = 16

# configure linear probe training
embedding_dim = 384 # change this for other architectures
hidden_dim = 256 # >0 for optimizing a second linear layer
num_classes_AS = 3
lr = 0.0001
num_epochs = 30

In [3]:
# load the backbone model, ensure params are consistent with ckpt
experiment = 'imagenet'
if experiment == 'imagenet':
    ckpt_path = None
    lora_rank = 0
elif experiment == 'full':
    ckpt_path = '../logs/training_base/checkpoint.pth'
    lora_rank = 0
elif experiment == 'lora4':
    ckpt_path = '../logs/training_1/checkpoint0009.pth'
    lora_rank = 4
else:
    raise ValueError()

arch = 'vit_small'
patch_size = 8
if ckpt_path == None:
    # load the default DINO model
    model = get_vit(arch, patch_size, lora_rank=0)
else:
    model = get_vit(arch, patch_size, lora_rank)
    load_lora_vit_from_dino_ckpt(model, ckpt_path)
model.to(device).eval()

Using cache found in ../pretrained_weights/facebookresearch_dino_main


Initialized without LoRA


Lora_vit(
  (base): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(8, 8), stride=(8, 8))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0-11): 12 x Block(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (norm): LayerNorm((384,

In [4]:
# load the dataset
transform = get_random_transform()
tr_dataset = TMED2(
    split = "train", # train/val/test/all/unlabeled
    transform = transform,
    parasternal_only = True,
    label_scheme_name = 'tufts',
)
tr_dataloader = torch.utils.data.DataLoader(tr_dataset, batch_size=batch_size, sampler = tr_dataset.class_sampler())

va_transform = get_deterministic_transform()
va_dataset = TMED2(
    split = "val", # train/val/test/all/unlabeled
    transform = va_transform,
    parasternal_only = True,
    label_scheme_name = 'tufts',
)
va_dataloader = torch.utils.data.DataLoader(va_dataset, batch_size=batch_size, shuffle=False)

[ 780  622 2444]
[0.00128205 0.00160772 0.00040917]


In [5]:
# instantiate trainable parameters, loss and optimizer

class Heads(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, num_classes_AS, **kwargs):
        super().__init__(**kwargs)
        if hidden_dim == 0:
            self.as_head = nn.Linear(embedding_dim, num_classes_AS)
            self.view_head = nn.Linear(embedding_dim, 2)
        else:
            self.as_head = nn.Sequential(nn.Linear(embedding_dim, hidden_dim), nn.ReLU(), nn.Linear(hidden_dim, num_classes_AS))
            self.view_head = nn.Sequential(nn.Linear(embedding_dim, hidden_dim), nn.ReLU(), nn.Linear(hidden_dim, 2))
    def forward(self, z):
        return self.as_head(z), self.view_head(z)
        
linear_probe = Heads(embedding_dim, hidden_dim, num_classes_AS).to(device)
# test the linear probe
print(linear_probe(torch.randn(batch_size, embedding_dim).to(device)))

tuner = torch.optim.Adam(linear_probe.parameters(), lr=lr)
loss_fcn = nn.CrossEntropyLoss(reduction="mean")

(tensor([[-0.1183, -0.0241, -0.2758],
        [ 0.1444, -0.0861, -0.1124],
        [-0.0420,  0.0587,  0.0451],
        [ 0.2020, -0.0809, -0.2038],
        [-0.3101, -0.3292, -0.1161],
        [ 0.5297, -0.0333, -0.1563],
        [ 0.3890, -0.0400,  0.1799],
        [ 0.1314, -0.0626,  0.1683],
        [-0.2073,  0.2356,  0.0367],
        [ 0.2517,  0.1855, -0.2147],
        [ 0.2597,  0.0150, -0.4146],
        [-0.1966, -0.0734, -0.0324],
        [ 0.1652, -0.3199, -0.2715],
        [ 0.0758,  0.1185,  0.0153],
        [ 0.0391,  0.0069,  0.0543],
        [-0.0601, -0.1089,  0.0041]], device='cuda:3',
       grad_fn=<AddmmBackward0>), tensor([[ 0.2966,  0.1669],
        [-0.0119,  0.1034],
        [-0.1613,  0.3144],
        [ 0.2857,  0.0116],
        [ 0.0330,  0.1729],
        [-0.2105,  0.0838],
        [ 0.0660,  0.3678],
        [-0.2163,  0.0780],
        [-0.3005,  0.0747],
        [-0.3021,  0.2450],
        [-0.0512, -0.1120],
        [-0.1931,  0.2655],
        [-0.1051,  

In [6]:
def forward(x):
    with torch.no_grad():
        features = model(x)
    logits_as, logits_v = linear_probe(features)
    return logits_as, logits_v
    
def train_batch(batch):
    x, [y, y_v] = batch
    x = x.to(device)
    y = y.to(device)
    y_v = y_v.to(device)

    tuner.zero_grad()
    logits_as, logits_v = forward(x)
    loss = loss_fcn(logits_as, y) + 0.1 * loss_fcn(logits_v, y_v)
    loss.backward()
    tuner.step()
    return loss.item()

def val_batch(batch):
    x, [y, y_v] = batch
    x = x.to(device)
    y = y.cpu().numpy()
    y_v = y_v.cpu().numpy()

    logits_as, logits_v = forward(x)
    pred_as = torch.argmax(logits_as, dim=1).cpu().numpy()
    pred_v = torch.argmax(logits_v, dim=1).cpu().numpy()
    return {'y_as':y, 'y_v':y_v, 'p_as':pred_as, 'p_v':pred_v}
            

In [7]:
for i in range(num_epochs):
    # train the model
    linear_probe.train()
    tr_loss = []
    for batch in tqdm(tr_dataloader):
        batch_loss = train_batch(batch)
        tr_loss.append(batch_loss)
    # validate the model
    linear_probe.eval()
    cache = {}
    for c in ['y_as', 'y_v', 'p_as', 'p_v']:
        cache[c] = []
    for batch in tqdm(va_dataloader):
        batch_outs = val_batch(batch)
        for k in batch_outs.keys():
            cache[k].extend(batch_outs[k])
    # evaluate acc and f1
    for k in cache.keys():
        cache[k] = np.array(cache[k]).squeeze()
    acc_as = sum(cache['y_as'] == cache['p_as'])/len(cache['y_as'])
    acc_v = sum(cache['y_v'] == cache['p_v'])/len(cache['y_v'])
    f1_as = f1_score(cache['y_as'], cache['p_as'], average='macro')
    f1_v = f1_score(cache['y_v'], cache['p_v'], average='macro')
    print("Epoch %3d: tr_loss %.3f, as acc/f1 %.3f/%.3f, view acc/f1 %.3f/%.3f" % (i, np.mean(tr_loss), acc_as, f1_as, acc_v, f1_v))
    print(confusion_matrix(cache['y_as'], cache['p_as']))
    print(confusion_matrix(cache['y_v'], cache['p_v']))

100%|█████████████████████████████████████████████████████████████████████████████████| 82/82 [00:17<00:00,  4.81it/s]


Epoch   0: tr_loss 0.990, as acc/f1 0.536/0.457, view acc/f1 0.914/0.885
[[ 93  90 103]
 [ 22 100  65]
 [123 205 509]]
[[925  40]
 [ 73 272]]


100%|█████████████████████████████████████████████████████████████████████████████████| 82/82 [00:17<00:00,  4.64it/s]


Epoch   1: tr_loss 0.890, as acc/f1 0.496/0.453, view acc/f1 0.921/0.898
[[176  47  63]
 [ 77  76  34]
 [315 124 398]]
[[909  56]
 [ 48 297]]


100%|█████████████████████████████████████████████████████████████████████████████████| 82/82 [00:11<00:00,  7.20it/s]


...

Epoch  19: tr_loss 0.547, as acc/f1 0.634/0.528, view acc/f1 0.934/0.910
[[154  44  88]
 [ 19  59 109]
 [114 105 618]]
[[948  17]
 [ 70 275]]


100%|█████████████████████████████████████████████████████████████████████████████████| 82/82 [00:18<00:00,  4.48it/s]


Epoch  20: tr_loss 0.540, as acc/f1 0.597/0.516, view acc/f1 0.934/0.909
[[157  52  77]
 [ 23  72  92]
 [123 161 553]]
[[951  14]
 [ 73 272]]


  0%|                                                                                         | 0/241 [00:00<?, ?it/s]


KeyboardInterrupt: 